In [125]:
# !pip install openpyxl

In [126]:
# ============================================================
# 🧭 Y06_안전비상벨_TM 주소 표준화 전처리 
# - ID 정리 (공백/탭 제거)
# - 중복 주소 컬럼 통합
# - 표준구주소 기준 시/구/동/도로명주소 분리 → INS_DETAIL에 반영
# - 코드형 컬럼 정제 (PNU/행정동/법정동/우편번호)
# - 표 출력 및 CSV 저장
# ============================================================

import pandas as pd
import re
from IPython.display import display

# ===== 0) 경로 =====
input_file  = "/Users/mac/Documents/SORA_Project/data/raw/Y06_안전비상벨_TM.csv"
output_file = "Y06_안전비상벨_TM_전처리.csv"

# ===== 1) 파일 로드 + NaN 확인 =====
df = pd.read_csv(input_file, sep="\t", encoding="utf-8-sig")
print(f"✅ 파일 로드 완료: {df.shape}\n")
print("📊 데이터 기본 정보 (NaN 확인 포함):")
df.info()  # 결측치 확인용


✅ 파일 로드 완료: (1000, 39)

📊 데이터 기본 정보 (NaN 확인 포함):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 39 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   _OBJT_ID    1000 non-null   object 
 1   FCLTY_TY    908 non-null    object 
 2   MNG_INST    908 non-null    object 
 3   INS_PURPOS  908 non-null    object 
 4   INS_TYPE    908 non-null    object 
 5   INS_DETAIL  908 non-null    object 
 6   RN_ADRES    908 non-null    object 
 7   ADRES       908 non-null    object 
 8   LAT         908 non-null    float64
 9   LON         908 non-null    float64
 10  LNK_TYPE    908 non-null    object 
 11  FLAG_POL_L  908 non-null    object 
 12  FLAG_SEC_L  908 non-null    object 
 13  FLAG_MNG_L  908 non-null    object 
 14  ADDITION    908 non-null    object 
 15  INS_YEAR    908 non-null    float64
 16  LAST_INSPD  908 non-null    float64
 17  LAST_INSPT  908 non-null    object 
 18  MNG_TEL     908 non-

In [133]:
# ===== 2) 항목정의 기반 컬럼 매핑 =====
col_map = {
    "OBJT_ID": "일련번호",
    "FCLTY_TY": "유형명",
    "MNG_INST": "관리기관명",
    "INS_PURPOS": "설치목적",
    "INS_TYPE": "설치장소유형",
    "INS_DETAIL": "설치위치",
    "RN_ADRES": "도로명주소",
    "ADRES": "지번주소",
    "LAT": "위도",
    "LON": "경도",
    "LNK_TYPE": "연계방식",
    "FLAG_POL_L": "경찰연계유무",
    "FLAG_SEC_L": "경비업체연계유무",
    "FLAG_MNG_L": "관리사무소연계유무",
    "ADDITION": "부가기능",
    "INS_YEAR": "안전비상벨설치연도",
    "LAST_INSPD": "최종점검일자",
    "LAST_INSPT": "최종점검결과구분",
    "MNG_TEL": "관리기관전화번호",
    "FLAG_SERVI": "연계유무",
    "CTPRVN_CD": "시도코드",
    "SGG_CD": "시군구코드",
    "EMD_CD": "읍면동코드",
    "X": "X좌표",
    "Y": "Y좌표",
    "DATA_TY": "데이터기준일자"
}

existing_map = {k: v for k, v in col_map.items() if k in df.columns}
df.rename(columns=existing_map, inplace=True)

print("\n✅ 컬럼명 한글 변환 완료")
print(df.columns.tolist())

# ===== 3) 숫자형 컬럼 처리 =====
float_cols = ["위도", "경도", "X좌표", "Y좌표", "안전비상벨설치연도"]
for col in float_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# ===== 4) 좌표 자릿수 고정 =====
def format_coords(x, decimals):
    if pd.isna(x):
        return None
    try:
        return round(float(x), decimals)
    except:
        return x

if "위도" in df.columns:
    df["위도"] = df["위도"].apply(lambda x: format_coords(x, 6))
if "경도" in df.columns:
    df["경도"] = df["경도"].apply(lambda x: format_coords(x, 6))
if "X좌표" in df.columns:
    df["X좌표"] = df["X좌표"].apply(lambda x: format_coords(x, 4))
if "Y좌표" in df.columns:
    df["Y좌표"] = df["Y좌표"].apply(lambda x: format_coords(x, 5))

# ===== 5) 코드 컬럼 문자열 변환 =====
code_cols = ["시도코드", "시군구코드", "읍면동코드"]
def clean_code_str(x):
    if pd.isna(x):
        return None
    s = str(x).strip()
    s = re.sub(r"[^\d]", "", s)
    return s if s else None

for c in code_cols:
    if c in df.columns:
        df[c] = df[c].apply(clean_code_str)

# ===== 6) 행정동명 기준 주소 파싱 =====
if "행정동명" not in df.columns:
    raise ValueError("❌ '행정동명' 컬럼이 없습니다.")

def parse_from_adm(addr):
    if pd.isna(addr):
        return None, None, None, None
    parts = re.split(r"\s+", str(addr).strip())
    si   = parts[0] if len(parts) > 0 else None
    gu   = parts[1] if len(parts) > 1 and re.search(r"(시|군|구)$", parts[1]) else None
    dong = parts[2] if len(parts) > 2 else None
    road = " ".join(parts[3:]) if len(parts) > 3 else None
    return si, gu, dong, road

df[["시", "구", "동", "도로명"]] = df["행정동명"].apply(lambda x: pd.Series(parse_from_adm(x)))

# ===== 7) NaN 보완 - 도로명주소 / 지번주소 / 좌표 기반 =====
def fill_address_from_row(row):
    # 도로명주소에서 채우기
    if pd.notna(row.get("도로명주소")):
        parts = str(row["도로명주소"]).strip().split()
        if len(parts) >= 3:
            if pd.isna(row["시"]): row["시"] = parts[0]
            if pd.isna(row["구"]): row["구"] = parts[1]
            if pd.isna(row["동"]): row["동"] = parts[2]
            if pd.isna(row["도로명"]) and len(parts) > 3:
                row["도로명"] = " ".join(parts[3:])

    # 지번주소에서 보완
    if pd.notna(row.get("지번주소")):
        parts = str(row["지번주소"]).strip().split()
        if len(parts) >= 3:
            if pd.isna(row["시"]): row["시"] = parts[0]
            if pd.isna(row["구"]): row["구"] = parts[1]
            if pd.isna(row["동"]): row["동"] = parts[2]
            if pd.isna(row["도로명"]) and len(parts) > 3:
                row["도로명"] = " ".join(parts[3:])

    return row

df = df.apply(fill_address_from_row, axis=1)

# 좌표 기반으로 보완
coord_lookup = df.dropna(subset=["시", "구", "동", "도로명"]).set_index(["위도", "경도"])[["시", "구", "동", "도로명"]]

def fill_from_coords(row):
    if (pd.isna(row["시"]) or pd.isna(row["구"]) or pd.isna(row["동"]) or pd.isna(row["도로명"])) \
        and (row["위도"], row["경도"]) in coord_lookup.index:
        vals = coord_lookup.loc[(row["위도"], row["경도"])]
        if pd.isna(row["시"]): row["시"] = vals["시"]
        if pd.isna(row["구"]): row["구"] = vals["구"]
        if pd.isna(row["동"]): row["동"] = vals["동"]
        if pd.isna(row["도로명"]): row["도로명"] = vals["도로명"]
    return row

df = df.apply(fill_from_coords, axis=1)

# ===== 8) 시/구/동/도로명 컬럼을 '유형명' 다음으로 이동 =====
target_cols = ["시", "구", "동", "도로명"]
target_cols = [c for c in target_cols if c in df.columns]
cols = list(df.columns)

if "유형명" in cols:
    type_idx = cols.index("유형명")
else:
    type_idx = 0

cols = [c for c in cols if c not in target_cols]
new_cols = cols[:type_idx+1] + target_cols + cols[type_idx+1:]
df = df[new_cols]

print("\n✅ 시/구/동/도로명 컬럼을 '유형명' 다음으로 이동 완료")
print(df.columns.tolist())

# ===== 9) 저장 =====
df.to_csv(output_file, index=False, encoding="utf-8-sig")
print(f"\n💾 전처리 완료 CSV 저장: {output_file}")

# ===== 10) 결과 확인 =====
print("\n🧾 결과 컬럼:")
print(df.columns.tolist())
display(df.head(10))


✅ 컬럼명 한글 변환 완료
['_OBJT_ID', '유형명', '시', '구', '동', '도로명', '관리기관명', '설치목적', '설치장소유형', '설치위치', '도로명주소', '지번주소', '위도', '경도', '연계방식', '경찰연계유무', '경비업체연계유무', '관리사무소연계유무', '부가기능', '안전비상벨설치연도', '최종점검일자', '최종점검결과구분', '관리기관전화번호', '연계유무', '시도코드', '시군구코드', '읍면동코드', 'X좌표', 'Y좌표', '데이터기준일자', '입력주소', 'X.1', 'Y.1', 'CLSS', 'PNU', '주소구분', '표준신주소', '표준구주소', '우편번호', '행정동코드', '행정동명', '법정동코드', '법정동명']


/var/folders/yt/2r0z55_978z15gtx5djp1y1w0000gn/T/ipykernel_80104/75550062.py:121: PerformanceWarning: indexing past lexsort depth may impact performance.
  and (row["위도"], row["경도"]) in coord_lookup.index:



✅ 시/구/동/도로명 컬럼을 '유형명' 다음으로 이동 완료
['_OBJT_ID', '유형명', '시', '구', '동', '도로명', '관리기관명', '설치목적', '설치장소유형', '설치위치', '도로명주소', '지번주소', '위도', '경도', '연계방식', '경찰연계유무', '경비업체연계유무', '관리사무소연계유무', '부가기능', '안전비상벨설치연도', '최종점검일자', '최종점검결과구분', '관리기관전화번호', '연계유무', '시도코드', '시군구코드', '읍면동코드', 'X좌표', 'Y좌표', '데이터기준일자', '입력주소', 'X.1', 'Y.1', 'CLSS', 'PNU', '주소구분', '표준신주소', '표준구주소', '우편번호', '행정동코드', '행정동명', '법정동코드', '법정동명']

💾 전처리 완료 CSV 저장: Y06_안전비상벨_TM_전처리.csv

🧾 결과 컬럼:
['_OBJT_ID', '유형명', '시', '구', '동', '도로명', '관리기관명', '설치목적', '설치장소유형', '설치위치', '도로명주소', '지번주소', '위도', '경도', '연계방식', '경찰연계유무', '경비업체연계유무', '관리사무소연계유무', '부가기능', '안전비상벨설치연도', '최종점검일자', '최종점검결과구분', '관리기관전화번호', '연계유무', '시도코드', '시군구코드', '읍면동코드', 'X좌표', 'Y좌표', '데이터기준일자', '입력주소', 'X.1', 'Y.1', 'CLSS', 'PNU', '주소구분', '표준신주소', '표준구주소', '우편번호', '행정동코드', '행정동명', '법정동코드', '법정동명']


,_OBJT_ID,유형명,시,구,동,도로명,관리기관명,설치목적,설치장소유형,설치위치,...,CLSS,PNU,주소구분,표준신주소,표준구주소,우편번호,행정동코드,행정동명,법정동코드,법정동명
0,1,비상벨,청운효자동,종로구,청운동,7-3,종로구청,약자보호,화장실,인왕산도시자연공원(청운지구 서시정),...,정좌표,1.111010e+18,구주소,NaN,-,NaN,1.111052e+09,청운효자동,1.111010e+09,청운동
1,2,비상벨,청운효자동,종로구,청운동,7-3,종로구청,방범용,공원,인왕산도시자연공원,...,정좌표,1.111010e+18,구주소,NaN,-,NaN,1.111052e+09,청운효자동,1.111010e+09,청운동
2,3,비상벨,서울특별시,종로구,청운동,7-27,종로구청,약자보호,화장실,인왕산도시자연공원(청운지구),...,정좌표,1.111010e+18,구주소,NaN,-,NaN,NaN,NaN,1.111010e+09,청운동
3,4,비상벨,청운효자동,종로구,청운동,산 1-1,종로구청,약자보호,화장실,창의문화장실,...,정좌표,1.111010e+18,구주소,서울특별시 종로구 창의문로 42,-,3048.0,1.111052e+09,청운효자동,1.111010e+09,청운동
4,5,비상벨,청운효자동,종로구,청운동,산 1-1,종로구청,약자보호,화장실,창의문화장실,...,정좌표,1.111010e+18,구주소,서울특별시 종로구 창의문로 42,-,3048.0,1.111052e+09,청운효자동,1.111010e+09,청운동
5,6,비상벨,청운효자동,종로구,궁정동,55-3,종로구청,약자보호,화장실,무궁화동산,...,정좌표,1.111010e+18,구주소,NaN,-,NaN,1.111052e+09,청운효자동,1.111010e+09,궁정동
6,7,비상벨,청운효자동,종로구,누상동,산 1-3,종로구청 청소행정과,방범용,화장실,누상동체육시설 공중화장실,...,정좌표,1.111011e+18,구주소,서울특별시 종로구 옥인6길 26-17,-,3038.0,1.111052e+09,청운효자동,1.111011e+09,누상동
7,8,비상벨,청운효자동,종로구,누상동,산 1-38,종로구청,약자보호,화장실,인왕산도시자연공원(누상지구),...,정좌표,1.111011e+18,구주소,NaN,-,NaN,1.111052e+09,청운효자동,1.111011e+09,누상동
8,9,비상벨,서울특별시,종로구,옥인동,185-4,종로구청,방범용,공원,수성동계곡,...,정좌표,1.111011e+18,구주소,NaN,-,NaN,NaN,NaN,1.111011e+09,옥인동
9,10,비상벨,청운효자동,종로구,옥인동,179-1,종로구청 청소행정과,방범용,화장실,인왕산수목원약수터 공중화장실,...,정좌표,1.111011e+18,구주소,NaN,-,3034.0,1.111052e+09,청운효자동,1.111011e+09,옥인동
